<a href="https://colab.research.google.com/github/toufiqmusah/IndabaX25/blob/main/Neural%20Architecture%20Search%20%26%20Deployment%20Optimization%20-%20Part%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial Title**
# **Efficient Deep Learning: Neural Architecture Search & Optimized Model Deployment**

# **Table of Contents**

1.   [Introduction](#Introduction)
2.   [Prerequisites](#Prerequisites)
3.   [Step-by-Step-Guide](#Step-by-Step-Guide)
4.   [Code Examples](#Code-Examples)
5.   [Troubleshooting](#Troubleshooting)
6.   [Conclusion](#Conclusion)
7.   [References](#References)

# 1. **Neural Architecture Search**

In [ ]:
!pip -q install keras-tuner

In [9]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [11]:
# model-building function that KerasTuner can use

def model_builder(hp):
    inputs = tf.keras.Input(shape=(32,32,3))
    x = inputs
    num_blocks = hp.Int('num_blocks', 2, 4)
    for i in range(num_blocks):
        filters = hp.Choice(f'filters_{i}', [24,32,40,48,64])
        kernel  = hp.Choice(f'kernel_{i}', [3,5])
        if hp.Boolean(f'ds_sep_{i}'):
            x = tf.keras.layers.SeparableConv2D(filters, kernel,
                                                padding='same', activation='relu')(x)
        else:
            x = tf.keras.layers.Conv2D(filters, kernel,
                                       padding='same', activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(rate=hp.Float('dropout',0.0,0.3,0.05))(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    lr = hp.Choice('lr',[1e-3, 5e-4, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model



# instantiating tuner

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=2,
                     factor=3,
                     directory='nas_dir',
                     project_name='intro_to_NAS')

In [ ]:
# adding early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# run NAS
print("Starting NAS search...")
tuner.search(x_train[:100], y_train[:100], epochs=15, validation_data=(x_test[:20], y_test[:20]), callbacks=[stop_early])
print("NAS search complete.")

# optimal hyperparameters and the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
nas_model = tuner.hypermodel.build(best_hps)
print(f"Best hyperparameters: {best_hps.values}")

In [ ]:
# train best model found by NAS

print("Training the best model found by NAS...")
history = nas_model.fit(x_train[:200], y_train[:200], epochs=50, validation_data=(x_test[:40], y_test[:40]))
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

In [ ]:
# re-initialize the model and train up to the best epoch

hypermodel = tuner.hypermodel.build(best_hps)
print("Retraining the best model until the best epoch...")
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_data=(x_test, y_test))
nas_best_model = hypermodel
print("Best model from NAS is trained and ready.")

# save model
nas_best_model.save("nas_optimal_model.keras")

Retraining the best model until the best epoch...
Epoch 1/22
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 186s 116ms/step - accuracy: 0.4003 - loss: 1.7295 - val_accuracy: 0.4371 - val_loss: 1.7215
Epoch 2/22
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 190s 108ms/step - accuracy: 0.6240 - loss: 1.0592 - val_accuracy: 0.6202 - val_loss: 1.0790
Epoch 3/22
